In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/local/openjdk-8"
os.environ["SPARK_HOME"] = "/user_data/spark-3.3.0-bin-hadoop2"

import findspark
findspark.init('spark-3.3.0-bin-hadoop2')

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = (
    SparkSession.builder.appName("ac2")
    .config("spark.sql.warehouse.dir", "hdfs:///user/hive/warehouse")
    .config("spark.sql.catalogImplementation", "hive")
    .getOrCreate()
)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/user_data/spark-3.3.0-bin-hadoop2/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/hadoop-2.7.3/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/27 01:34:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [47]:
dataframe = spark.read.csv("hdfs://spark-master:9000/datasets/flight/Combined_Flights_2022.csv", header=True, inferSchema=True)
use_cols = [
    'Month',
    'FlightDate',
    'Airline',
    'Origin',
     'Dest',
     'Cancelled',
     'Diverted',
     'CRSDepTime',
     'DepTime',
     'DepDelayMinutes', 
     'OriginAirportID',
     'OriginStateName',
     'DestAirportID',
     'DestCityName',
     'DestStateName',
    'Marketing_Airline_Network',
    'ArrDelayMinutes'
]
dataframe = dataframe.select(*use_cols)
lines_count = dataframe.count()
print(f"Número de linhas no DataFrame: {lines_count}")

Número de linhas no DataFrame: 4078318


In [22]:
dataframe.printSchema()

root
 |-- FlightDate: timestamp (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DestCityName: string (nullable = true)
 |-- DestStateName: string (nullable = true)



In [23]:
dataframe.show(5)

+-------------------+--------------------+------+----+---------+--------+----------+-------+---------------+---------------+---------------+-------------+-------------------+-------------+
|         FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|OriginAirportID|OriginStateName|DestAirportID|       DestCityName|DestStateName|
+-------------------+--------------------+------+----+---------+--------+----------+-------+---------------+---------------+---------------+-------------+-------------------+-------------+
|2022-04-04 00:00:00|Commutair Aka Cha...|   GJT| DEN|    false|   false|      1133| 1123.0|            0.0|          11921|       Colorado|        11292|         Denver, CO|     Colorado|
|2022-04-04 00:00:00|Commutair Aka Cha...|   HRL| IAH|    false|   false|       732|  728.0|            0.0|          12206|          Texas|        12266|        Houston, TX|        Texas|
|2022-04-04 00:00:00|Commutair Aka Cha...|   DRO| DEN| 

Como o número de células vazias no nosso dataset é de cerca de 3%, o que é uma quantidade baixa, iremos apenas eliminar esses dados, ao invés de tratarmos.

In [24]:
total_count = dataframe.count()
not_null_count = dataframe.dropna().count() 
percentage_not_null = not_null_count * 100 / total_count
print(f"Porcentagem não nula: {percentage_not_null}")
dataframe = dataframe.dropna()

Porcentagem não nula: 97.04547315829713


- Analisando os voos cancelados, foi observado que 3% da base de dados foram cancelados.
- O mês de janeiro (1) apresentou a maior quantidade de voos cancelados, em seguida de fevereiro (2) e junho (6)
- A companhia áerea AA, seguida da UA e DL apresentaram maior quantidades de voos cancelados.

In [51]:
# Proporção de voos cancelados
cancelled_ratio = (
    dataframe.select(
        F.count(
            F.when(
                F.col("Cancelled") == "True", 1
            )
        )
    ).collect()[0][0] / dataframe.count()
) * 100

print(f"Proporção de Voos Cancelados: {cancelled_ratio:.2f}%")

# Distribuição de voos cancelados por mês
df_cancelado_por_mes = dataframe.groupBy("Month").agg(
    F.count(
        F.when(
            F.col("Cancelled") == "True", 1
        )
    ).alias("Voos cancelados")
).orderBy("Month")
df_cancelado_por_mes.show()

# Distribuição de voos cancelados por companhia aérea
cancelled_by_network = dataframe.groupBy("Marketing_Airline_Network").agg(
    F.count(
        F.when(
            F.col("Cancelled") == "True", 1
        )
    ).alias("Voos cancelados")
).orderBy("Voos cancelados")
cancelled_by_network.show()

Proporção de Voos Cancelados: 3.02%


+-----+---------------+
|Month|Voos cancelados|
+-----+---------------+
|    1|          35667|
|    2|          23421|
|    3|           9108|
|    4|          13397|
|    5|          11993|
|    6|          18473|
|    7|          11133|
+-----+---------------+



+-------------------------+---------------+
|Marketing_Airline_Network|Voos cancelados|
+-------------------------+---------------+
|                       HA|            351|
|                       F9|           2501|
|                       G4|           2891|
|                       NK|           4198|
|                       AS|           5016|
|                       B6|           7631|
|                       WN|          18164|
|                       DL|          20969|
|                       UA|          24705|
|                       AA|          36766|
+-------------------------+---------------+



- Analisando o tempo de atraso, foi observado uma média de 15 minutos de atraso.
- O mês de julho (7) e junho (6) apresentaram um tempo médio de atraso maior
- As companias B6, G4 e F9 apresentaram maior tempo de atraso médio

In [52]:
# Tempo médio de atraso (minutos)
average_delay = dataframe.select(F.avg("ArrDelayMinutes")).collect()[0][0]
print(f"Tempo Médio de Atraso: {average_delay:.2f} minutos")

# Distribuição de atrasos por mês
delay_by_month = dataframe.groupBy("Month").agg(F.avg("ArrDelayMinutes").alias("atraso_medio")).orderBy("Month")
delay_by_month.show()

# Distribuição de atrasos por companhia aérea
delay_by_company = dataframe.groupBy("Marketing_Airline_Network").agg(F.avg("ArrDelayMinutes").alias("atraso_medio")).orderBy("atraso_medio", ascending=False)
delay_by_company.show()

Tempo Médio de Atraso: 15.78 minutos


+-----+------------------+
|Month|      atraso_medio|
+-----+------------------+
|    1|14.354921483258492|
|    2| 14.22176289364896|
|    3|15.282290116612899|
|    4|15.949220248947393|
|    5|15.079478974372895|
|    6|17.849852879505566|
|    7|17.326682130740213|
+-----+------------------+



+-------------------------+------------------+
|Marketing_Airline_Network|      atraso_medio|
+-------------------------+------------------+
|                       B6| 28.37808613922048|
|                       G4| 25.35006813149378|
|                       F9| 23.98092831430138|
|                       NK|19.172122475152293|
|                       AA|16.423352661730473|
|                       UA|16.016196415089063|
|                       WN|13.865528202334586|
|                       DL| 13.74481651165211|
|                       AS|10.575677971939655|
|                       HA|  9.06934315296143|
+-------------------------+------------------+



In [25]:
allowed_types = ['int', 'double']

# Filtrando colunas com tipos permitidos
selected_columns = [col_name for col_name, col_type in dataframe.dtypes if any(data_type in col_type for data_type in allowed_types)]

# Selecionando apenas as colunas permitidas
df_filtered = dataframe.select(*selected_columns)

# Mostrando o esquema do DataFrame resultante
df_filtered.printSchema()

root
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)



# Pré processamento

In [26]:
target = "O"
feature_columns = df_filtered.columns
feature_columns.remove(target)
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_vector = vector_assembler.transform(df_filtered)

ValueError: list.remove(x): x not in list

In [ ]:
# Dividindo o conjunto de dados entre treino e teste
train_data, test_data = df_vector.randomSplit(
    [0.8, 0.2], # 80% treino e 20% teste
    seed=42 # Semente de aleatoriedade
)

# Model

In [ ]:
rf_classifier = RandomForestClassifier(labelCol=target, featuresCol="features", numTrees=10)
pipeline = Pipeline(stages=[rf_classifier])
model = pipeline.fit(train_data)
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="Severity4", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)
